In [1]:
import numpy as np
import matplotlib.pyplot as plt
from TemporalNetwork import ContTempNetwork, StaticTempNetwork
from scipy.sparse import (lil_matrix, dok_matrix, diags, eye, isspmatrix_csr, isspmatrix,
                          csr_matrix, coo_matrix, csc_matrix)
import compute_S_rate
import auxiliary_functions
import seaborn
import math

import networkx as nx

Could not load sparse_dot_mkl. Will use scipy.sparse for matrix products.


In [2]:
adj_graph = np.array([[0, 1, 1, 1], [1, 0, 1, 0], [1, 1, 0, 0], [1, 0, 0, 0]])

In [3]:
time = [i for i in np.linspace(1,25,3)]
graph= StaticTempNetwork(times = time, adjacency = csr_matrix(adj_graph))
graph.compute_laplacian_matrices(random_walk = True)

graph.compute_inter_transition_matrices(lamda=1, dense_expm=False, use_sparse_stoch=False)
graph.compute_transition_matrices(lamda=1)

/home/b/skoove/Desktop/entropy/entropy_env/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/home/b/skoove/Desktop/entropy/entropy_env/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:302: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


## Uniform initial ditribution

In [4]:
H_graph = compute_S_rate.compute_conditional_entropy(net=graph, list_T=graph.T[1], lamda=1, force_csr=True, time_domain= list(range(len(time)-2,len(time)-1)))

In [5]:
p0 = 1/4*np.ones(4)
T = graph.T[1][-1].toarray() 
print(p0 @ T)

[0.375 0.25  0.25  0.125]


In [6]:
Tcsr = csr_matrix(T)
logTdata = np.log(np.where(Tcsr.data > 0, Tcsr.data, 1))
print(logTdata)
TlogTdata = Tcsr.data * logTdata
print(TlogTdata)
TlogT = csr_matrix((TlogTdata, Tcsr.indices, Tcsr.indptr), shape=T.shape)
print(TlogT.toarray())
print(-np.sum(p0 @ TlogT, where= np.isfinite(p0 @ TlogT)))

[-0.98082925 -1.38629436 -1.38629436 -2.07944153 -0.98082926 -1.38629435
 -1.38629435 -2.07944156 -0.98082926 -1.38629435 -1.38629435 -2.07944156
 -0.98082924 -1.38629438 -1.38629438 -2.0794415 ]
[-0.36781097 -0.34657359 -0.34657359 -0.25993019 -0.36781097 -0.34657359
 -0.34657359 -0.25993019 -0.36781097 -0.34657359 -0.34657359 -0.25993019
 -0.36781097 -0.34657359 -0.34657359 -0.2599302 ]
[[-0.36781097 -0.34657359 -0.34657359 -0.25993019]
 [-0.36781097 -0.34657359 -0.34657359 -0.25993019]
 [-0.36781097 -0.34657359 -0.34657359 -0.25993019]
 [-0.36781097 -0.34657359 -0.34657359 -0.2599302 ]]
1.3208883433450418


In [7]:
n_edges = adj_graph.sum() / 2
print(n_edges)
degrees = adj_graph.sum(0)
print(degrees)
avg_degree = np.mean(degrees)
print(avg_degree)
avg_degreelogdegree = np.mean(degrees * np.log(degrees))
print(avg_degreelogdegree)
asymptot = np.log(2 * n_edges) - avg_degreelogdegree / avg_degree
print(asymptot)

4.0
[3 2 2 1]
2.0
1.5171063970610275
1.320888343149322


In [44]:
list_expected_approx = []
n = 4
p = 0.3
k_avg = (3) * p

approx_asymptot = np.log(2* math.comb(n, 2)* p) - np.log(k_avg)

In [45]:
print(H_graph[list(H_graph.keys())[0]][-1])
print(asymptot)
print(approx_asymptot)

1.3208883433450418
1.320888343149322
1.3862943611198906


## RW diffusion - Uniform distribution

In [10]:
time = [i for i in np.linspace(1,100,2)]
N = [10, 50, 100]
n_samples = 10
p= 0.2

list_H = []
list_asymptot = []

for i, n in enumerate(N):
    er_adj = nx.adjacency_matrix(nx.erdos_renyi_graph(n, p))
    er_rw = StaticTempNetwork(times = time, adjacency = csr_matrix(er_adj))
    er_rw.compute_laplacian_matrices(random_walk = True)

    er_rw.compute_inter_transition_matrices(lamda=1, dense_expm=False, use_sparse_stoch=False)
    er_rw.compute_transition_matrices(lamda=1)

    H_er_rw = compute_S_rate.compute_conditional_entropy(net=er_rw, list_T=er_rw.T[1], lamda=1, force_csr=True, time_domain= list(np.arange(len(time)-2, len(time)-1)))
    list_H.append(H_er_rw[list(H_er_rw.keys())[0]][-1])
    
    n_edges = er_adj.toarray().sum() / 2
    degrees = er_adj.toarray().sum(0)
    avg_degree = np.mean(degrees)
    avg_degreelogdegree = np.mean(degrees *np.log(degrees))
    list_asymptot.append(np.log(2 * n_edges) - avg_degreelogdegree / avg_degree)

In [11]:
print(list_H)
print(list_asymptot)

[2.250260278618416, 3.874389873139027, 4.582153396303933]
[2.250260278617591, 3.874389873139023, 4.58215339630393]


## RW-Diffusion - Stationary distribution

We know that the asymptotic value should be independent of the initial ditribution. We can check if the property holds to test if our implementation of the conditional entropy works for different starting distributions.

In [48]:
time = [i for i in np.linspace(1,100,2)]
N = [10, 50, 100, 500]
n_samples = 10
p= 0.2

list_H = []
list_asymptot = []

for i, n in enumerate(N):
    er_adj = nx.adjacency_matrix(nx.erdos_renyi_graph(n, p))
    er_rw = StaticTempNetwork(times = time, adjacency = csr_matrix(er_adj))
    er_rw.compute_laplacian_matrices(random_walk = True)

    er_rw.compute_inter_transition_matrices(lamda=1, dense_expm=False, use_sparse_stoch=False)
    er_rw.compute_transition_matrices(lamda=1)

    n_edges = er_adj.toarray().sum() / 2
    degrees = er_adj.toarray().sum(0)
    pi_0 = degrees / (2*n_edges)
    
    H_er_rw = compute_S_rate.compute_conditional_entropy(net=er_rw, list_T=er_rw.T[1], lamda=1, force_csr=True, time_domain= list(np.arange(len(time)-2, len(time)-1)), p0 = pi_0)
    list_H.append(H_er_rw[list(H_er_rw.keys())[0]][-1])
    
    
    avg_degree = np.mean(degrees)
    corrected_degrees  = np.where(degrees > 0, degrees, 1)
    avg_degreelogdegree = np.mean(degrees * np.log(corrected_degrees))
    list_asymptot.append(np.log(2 * n_edges) - avg_degreelogdegree / avg_degree)

list_approx_asymptot = []
for n in N:
    p = 0.2
    k_avg = (n-1) * p
    list_approx_asymptot.append(np.log(2* math.comb(n, 2)* p) - np.log(k_avg))

/home/b/skoove/Desktop/entropy/entropy_env/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/home/b/skoove/Desktop/entropy/entropy_env/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:302: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


In [49]:
print(list_H)
print(list_asymptot)
print(list_approx_asymptot)

[2.2462138366774425, 3.8770521286362447, 4.588056188187807, 6.210243821325073]
[2.246213836677445, 3.8770521286362567, 4.588056188187809, 6.210243821325097]
[2.3025850929940455, 3.9120230054281455, 4.605170185988092, 6.214608098422191]
